In [1]:
import os
import sys
module_path_dont_know = os.path.abspath(os.path.join('../COBRAS_dont_know'))
module_path_testing = os.path.abspath(os.path.join('../COBRAS_testing'))

if module_path_dont_know not in sys.path:
    sys.path.append(module_path_dont_know)
    print("module path of dont_know added")

if module_path_testing not in sys.path:
    sys.path.append(module_path_testing)
    print("module path of testing added")
    
from pathlib import Path
from config import FOLD_RESULT_DIR, FIGURE_DIR
from before_clustering.generate_folds import generate_folds_for_dataset

from heuristics.splitlevel_estimation_strategy import *
from heuristics.select_super_instance_heuristics import *
from cobras.cobras import COBRAS
from util.datasets import Dataset
from cobras.querier.weak_querier import WeakQuerier
from cobras.querier.labelquerier import LabelQuerier
from clustering_algorithms.kmeans_fixed_representative import KmeansFixedRepresentative
from generate_clusterings.clustering_task import make_n_run_10_fold_cross_validation
from cobras.super_instances.superinstance_select_representative import SuperInstance_select_representative_Builder

from run_locally.run_tests import run_clustering_tasks_locally

from evaluate_clusterings.calculate_aris import calculate_n_times_n_fold_aris_for_testnames
from evaluate_clusterings.calculate_average_aris import calculate_average_aris
from evaluate_clusterings.calculate_aligned_rank import calculate_and_write_aligned_rank

from present_results.plot_aris import plot_average_ARI_per_dataset
from present_results.plot_aris import plot_overall_average_ARI
from present_results.plot_aligned_rank import plot_rank_comparison_file

print("imports succeeded")

module path of dont_know added
module path of testing added
imports succeeded


In [2]:
if not Path(FOLD_RESULT_DIR).exists():
    generate_folds_for_dataset()
print("done")

done


In [3]:
#data = Dataset("iris")
clusterer_1 = COBRAS(cluster_algo= KmeansFixedRepresentative())
querier = LabelQuerier(None,1000)
#querier = WeakQuerier(data.data,data.target,1000,'local_nondet')
#querier = WeakQuerier(None, None ,1000,'local_nondet')
test_name_1 = "test_fixed_k-means"
all_dataset_names = ["iris"]#Dataset.get_standard_dataset_names()#
clustering_tasks = make_n_run_10_fold_cross_validation(test_name_1, clusterer_1,querier,all_dataset_names,10)
print("done")

done


In [4]:
clusterer_2 = COBRAS(cluster_algo=KmeansFixedRepresentative(), splitlevel_strategy=StandardSplitLevelEstimationStrategyAlwayskmeans(SelectLeastInstancesHeuristic()))
test_name_2 = "test_Basic_k-Means"
clustering_tasks.extend(make_n_run_10_fold_cross_validation(test_name_2, clusterer_2, querier, all_dataset_names, 10))
print("done")

done


In [5]:
run_clustering_tasks_locally(clustering_tasks,4)
print("done")

 50%|█████     | 100/200 [00:47<00:47,  2.11it/s]


TypeError: 'NoneType' object is not iterable

In [ ]:
test_names = [test_name_1, test_name_2]
comparison_name = 'test_splitlevel_estimation_fixed_k-means_vs_basic_k-means'
nb_of_cores = 4
query_budget = 250

calculate_n_times_n_fold_aris_for_testnames(test_names, nb_cores=nb_of_cores)
calculate_average_aris(test_names, query_budget)
calculate_and_write_aligned_rank(test_names,comparison_name)

plot_average_ARI_per_dataset(comparison_name, test_names, test_names)
plot_overall_average_ARI(comparison_name, test_names, test_names)
plot_rank_comparison_file(comparison_name, test_names, test_names)

print(f"all resulting plots are in {Path(FIGURE_DIR)/comparison_name}")